In [1]:
import onnx
import onnxruntime
import numpy as np
import cv2
from PIL import Image
import torch

In [38]:
session = onnxruntime.InferenceSession('average_model.onnx')
image_raw = cv2.imread('moto.jpeg')
image = cv2.resize(np.array(image_raw), (640,640), interpolation = cv2.INTER_LINEAR)
image = np.transpose(image, (2,0,1))

image = np.expand_dims(image, axis=0)


image.shape

(1, 3, 640, 640)

In [34]:
input_name = session.get_inputs()[0].name
print("Input name  :", input_name)
input_shape = session.get_inputs()[0].shape
print("Input shape :", input_shape)
input_type = session.get_inputs()[0].type
print("Input type  :", input_type)

output_name = session.get_outputs()[0].name
print("Output name  :", output_name)
output_shape = session.get_outputs()[0].shape
print("Output shape :", output_shape)
output_type = session.get_outputs()[0].type
print("Output type  :", output_type)
output_name = session.get_outputs()[1].name
print("Output name  :", output_name)
output_shape = session.get_outputs()[1].shape
print("Output shape :", output_shape)
output_type = session.get_outputs()[1].type
print("Output type  :", output_type)

Input name  : onnx::Cast_0
Input shape : [1, 3, 640, 640]
Input type  : tensor(uint8)
Output name  : pre_nms_bboxes_xyxy
Output shape : [1, 1000, 4]
Output type  : tensor(float)
Output name  : pre_nms_scores
Output shape : [1, 1000, 5]
Output type  : tensor(float)


In [39]:
import time

times=[]
for i in range(1,50):
    t0 = time.time()
    results = session.run(None, {'onnx::Cast_0':image})
    t1 = time.time()
    times.append(1/(t1-t0))
    
print(times)

[5.301367335124764, 5.23847967707126, 5.425649795421001, 4.4850088164562445, 4.4511013926459215, 4.490910687250121, 4.196162900164773, 4.5265236788883305, 4.312882388386293, 4.5153887808149715, 4.788242372074492, 4.8907177546772695, 4.762168821068288, 4.865302145380096, 4.844090268634652, 4.627686875930932, 4.342046473300524, 4.559710043463041, 4.514980489248903, 4.37568032515813, 4.478285519658078, 4.389601772458449, 4.503418656588215, 4.61643547639724, 4.593537534169762, 4.657863934488128, 4.688914104291152, 4.000862307435494, 4.067689043153753, 4.752370064278114, 4.642906244223078, 3.5828265718149197, 3.556701275706625, 3.983073638787805, 3.9719164007234915, 3.437804751105075, 4.30918249616527, 4.759558711721464, 3.9380251962767048, 3.9477548653916927, 3.497386739244272, 3.4952038179459124, 4.225079478924486, 4.089410617657095, 3.8454381593109885, 3.7839019901484945, 3.610764705224038, 3.9902657510840673, 4.484021086414315]


In [41]:
vec = []
boxes = []
scores = []
import torch
sf = torch.nn.Softmax(dim=1)
for i in range(0,1000):
    xc,yc,w,h = results[0][0,i,:]
    out = list(results[1][0,i,:]) 
    boxes.append( (xc,yc,w,h) )
    scores.append(max(out))
    

In [42]:

def nms(bounding_boxes, confidence_score, threshold):
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes
    score = np.array(confidence_score)

    # Picked bounding boxes
    picked_boxes = []
    picked_score = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])

        # Compute ordinates of intersection-over-union(IOU)
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)

        left = np.where(ratio < threshold)
        order = order[left]

    return picked_boxes, picked_score

In [43]:
boxes_nms, scores_nms = nms(boxes, scores, 0.05)
len(boxes_nms)

26

In [44]:
from PIL import ImageDraw
from PIL import Image
image = Image.open('moto3.jpeg')
image = image.resize((640,640))
draw = ImageDraw.Draw(image)
for box in boxes_nms:
    x1,y1,x2,y2 = box[0],box[1],box[2],box[3]
    #print(f'{x1}_{y1}_{x2}_{y2}')
    bbox = [(x1,y1),(x2,y2)]
    draw.rectangle(bbox, outline="black")
    
image.show()